In [1]:
import os
from src.StreamPort.device.analyses import MassSpecAnalyses

path = "C:/Users/Sandeep/Desktop/Error-LC/Method-Data"
batches = os.listdir(path)
batches = [os.path.join(path, file) for file in batches]

error_lc_files = []
for batch in batches:
    batch_files = os.listdir(batch)
    batch_files = [os.path.join(batch, file) for file in batch_files if ".D" in file]
    error_lc_files.extend(batch_files)

analyses = MassSpecAnalyses(files=error_lc_files)
print("Number of analyses: ", len(analyses.data))

Number of analyses:  658


In [2]:
batches = analyses.get_batches()
print(batches)

['250616_Mix-1_50x100ng-mL 2025-06-16 10-06-17', '250707_Mix-1+IS_30x100ng-mL 2025-07-07 14-47-33', '250703_Mix-1+IS_20x100ng-mL 2025-07-03 09-29-09', '250703_Mix-1+IS_10x100ng-mL 2025-07-03 14-14-18', '250620_Mix-1_25x100ng-mL 2025-06-20 09-26-34', '250625_Mix-1_25x100ng-mL 2025-06-25 07-55-23', '250624_Mix-1_20x100ng-mL 2025-06-24 16-17-42', '250623_Mix-1_25x100ng-mL 2025-06-23 15-57-58', '250704_Mix-1+IS_25x100ng-mL 2025-07-04 10-01-22', '250702_Mix-1+IS_50x100ng-mL 2025-07-02 09-46-06', '250624_Mix-1_30x100ng-mL 2025-06-24 10-33-02', '250613_Mix-1_10x10ng-mL 2025-06-13 14-40-12', '250707_Mix-1+IS_20x100ng-mL 2025-07-07 08-54-14', '250626_Mix-1_50x100ng-mL 2025-06-26 09-05-34', '250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43', '250625_Mix-1_35x100ng-mL 2025-06-25 14-38-46', '250617_Mix-1_50x10ng-mL 2025-06-17 09-19-10']


In [ ]:
# only mzs available in MSD1: 120, 124, 134, 237, 247, 254, 258, 267, 268, 275, 278, 296, 300, 325, 370, 429, 441, 748, 752

In [5]:
chromas = analyses.plot_data(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8], mz=752) 
chromas.show()

In [ ]:
file = error_lc_files[6]
print(file)
### replace Error-LC/Method-Data\\250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43\\002-D2F-B1-Flush and 003-D2F-B2-Blank from server and run csv creation on them

import rainbow as rb
meta = rb.read_metadata(file)
print(meta)

datadir = rb.read(file)
print(datadir.metadata)

datafiles = datadir.datafiles
for f in datafiles: # xlabels= retention time, ylabels = mz/wavelength, data = intensity
    print(f.metadata.get("notebook"))
    print(f.get_info())

print(f.extract_traces()) # returns data (intensity) for the given ylabels. Rows correspond to ylabels

f.plot(label=325) # label represents the ylabel (mz) for which data should be plotted

C:/Users/Sandeep/Desktop/Error-LC/Method-Data\250613_Mix-1_10x100ng-mL 2025-06-13 17-02-43\007-D2F-A1-Mix 1.D
